# Capstone Project: Feature Selection and Engineering <br>

#### Brandie Hatch

###  Features

- Selection based on EDA
- Engineering
- Dimensionality Reduction


## Import Data


In [1]:
# python library imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as stats
import statsmodels.api as sm
from sklearn.feature_selection import VarianceThreshold
from sklearn.manifold import MDS

%matplotlib inline
pd.options.display.max_columns =999

c:\Users\hatch\anaconda3\lib\site-packages\statsmodels\compat\pandas.py:61: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import Int64Index as NumericIndex


In [2]:
# read in data
df = pd.read_csv("./data/data_clean.csv")

In [3]:
df.shape

(3239553, 26)

### T-test and Stats Model Logit Regression for Feature Engineering

__EDA Conclusion:__ <br>

Assuming all else is constant, the Logistic Regression Stats Model shows....


The log odds for 'SEX' prediction increases by 0.17 if the household is multigenerational ('MUTLG').

The log odds for 'SEX' prediction increases by 0.06 as the level of education attainment increases ('SCHL')

The log odds for 'SEX' prediction decreases by 


Look at original data because the p values of like sch should be lower than they are

Standard Scaler and one hot encoder before stats model og reg


Go back to fundamental data science - do a T test, to see if age / sex are really different? Is it really different than the other one
statistical way of actually figuring it out
T test is better - -

In [ ]:
dfs = df.sample(n=1000, replace=False, axis=0, ignore_index=True, random_state=42)
print(dfs.shape)
dfs.head()

(1000, 26)


,AGEP,CIT,COW,ENG,ESR,FFSP,LANX,MAR,MULTG,NOC,OCCP,PINCP,POVPIP,RAC1P,SCH,SCHG,SCHL,SCIENGP,SCIENGRLP,SEMP,SEX,WAGP,WKEXREL,WKL,WRK,STATE
0,49,1,0,0,6,0,2,3,1,2,9,27800,136,1,1,0,19,0,0,0,1,0,12,3,2,26
1,70,1,0,0,6,0,2,2,1,0,9,38000,442,1,1,0,13,0,0,0,1,0,12,3,2,42
2,58,1,1,0,1,0,2,1,1,0,410,89000,501,1,1,0,16,0,0,0,1,89000,1,1,1,36
3,76,1,0,0,6,0,2,2,1,0,9,96400,501,1,1,0,21,2,2,0,1,0,0,3,2,17
4,57,1,0,0,6,0,2,3,1,0,9,9700,73,1,1,0,16,0,0,0,1,0,0,3,2,39


In [ ]:
ts, p = stats.ttest_ind(df.loc[df['SEX']==1, 'AGEP'], df.loc[df['SEX']==2, 'AGEP'])
p
# significant p-value that shows age is significant in predicting sex

6.410025583474209e-05

In [ ]:
df['SEX'] = df['SEX'].map({1:0, 2:1})

In [ ]:
X = df.drop(columns=['SEX'])
y = df['SEX']

In [ ]:
log_reg = sm.Logit(y, X).fit()


# stats models logistic regression https://www.geeksforgeeks.org/logistic-regression-using-statsmodels/
# First try got ValueError: endog must be in the unit interval. Solution is to change the target column to 1 and 0 (currently it is 1 and 2) https://stackoverflow.com/questions/31322370/valueerror-endog-must-be-in-the-unit-interval

Optimization terminated successfully.
         Current function value: 0.655203
         Iterations 6


In [ ]:
print(log_reg.summary())

                           Logit Regression Results                           
Dep. Variable:                    SEX   No. Observations:                10000
Model:                          Logit   Df Residuals:                     9975
Method:                           MLE   Df Model:                           24
Date:                Mon, 15 Aug 2022   Pseudo R-squ.:                 0.05468
Time:                        16:36:50   Log-Likelihood:                -6552.0
converged:                       True   LL-Null:                       -6931.0
Covariance Type:            nonrobust   LLR p-value:                1.568e-144
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
AGEP           0.0040      0.002      2.360      0.018       0.001       0.007
CIT            0.0244      0.025      0.977      0.328      -0.025       0.073
COW           -0.0090      0.014     -0.622      0.5

### Feature Selection Exploration using Multidimensional Scaling and Variance Threshold

https://towardsdatascience.com/dimensionality-reduction-for-machine-learning-80a46c2ebb7e 

In [31]:
# X for MDS
X = df.drop(columns='SEX')

In [32]:
embedding = MDS(n_components=2, random_state=42)
X_transformed = embedding.fit_transform(X[:1000])

In [33]:
X_transformed.shape

(1000, 2)

In [ ]:
# X for VarianceThreshold
X = df.drop(columns='SEX')

In [14]:
sel = VarianceThreshold(threshold=(.8 * (1 - .8)))
sel.fit_transform(X)

array([[35,  1,  1, ...,  1,  0, 18],
       [25,  1,  0, ...,  3,  0, 48],
       [21,  5,  2, ...,  1,  0, 18],
       ...,
       [30,  1,  1, ...,  1,  1, 37],
       [26,  1,  1, ...,  1,  1, 48],
       [26,  1,  1, ...,  1,  1, 48]], dtype=int64)

In [15]:
X.shape

(3239553, 25)

## Feature Engineering

### States sorted by Census Regions and Divisions:

__Northeast Region__
- New England Division: Connecticut, Maine, Massachusetts, New Hampshire, Rhode Island and Vermont
- Middle Atlantic Division: New Jersey, New York and Pennsylvania<br>

__Midwest Region__
- East North Central Division: Illinois, Indiana, Michigan, Ohio and Wisconsin
- West North Central Division: Iowa, Kansas, Minnesota, Missouri, Nebraska, North Dakota and South Dakota<br>

__South Region__
- South Atlantic Division: Delaware, District of Columbia, Florida, Georgia, Maryland, North Carolina, South Carolina, Virginia and West Virginia
- East South Central Division: Alabama, Kentucky, Mississippi and Tennessee
- West South Central Division: Arkansas, Louisiana, Oklahoma and Texas<br>

__West Region__
- Mountain Division: Arizona, Colorado, Idaho, Montana, Nevada, New Mexico, Utah and Wyoming
- Pacific Division: Alaska, California, Hawaii, Oregon and Washington

Source: https://www.census.gov/programs-surveys/economic-census/guidance-geographies/levels.html#par_textimage_34

In [24]:
dfs = df.sample(n=100000, replace=False, axis=0, ignore_index=True, random_state=42)
print(dfs.shape)
dfs.head()

(100000, 26)


,AGEP,CIT,COW,ENG,ESR,FFSP,LANX,MAR,MULTG,NOC,OCCP,PINCP,POVPIP,RAC1P,SCH,SCHG,SCHL,SCIENGP,SCIENGRLP,SEMP,SEX,WAGP,WKEXREL,WKL,WRK,STATE
0,49,1,0,0,6,0,2,3,1,2,9,27800,136,1,1,0,19,0,0,0,1,0,12,3,2,26
1,70,1,0,0,6,0,2,2,1,0,9,38000,442,1,1,0,13,0,0,0,1,0,12,3,2,42
2,58,1,1,0,1,0,2,1,1,0,410,89000,501,1,1,0,16,0,0,0,1,89000,1,1,1,36
3,76,1,0,0,6,0,2,2,1,0,9,96400,501,1,1,0,21,2,2,0,1,0,0,3,2,17
4,57,1,0,0,6,0,2,3,1,0,9,9700,73,1,1,0,16,0,0,0,1,0,0,3,2,39


In [ ]:
# Create feature for US Regions: West, South, Midwest, Northeast
# Create feature for US Divisions
# can use groupby with the following new DataFrame

In [25]:
df_state = pd.read_excel("./data/states.xlsx", sheet_name='RegDivSort')
# df_state.rename(columns={'State': 'STATE_NM'}, inplace=True)

In [26]:
df_state.head()

,STATE,STATE_NM,ABBV,REGION,DIVISION
0,4,Arizona,AZ,West,Mountain
1,8,Colorado,CO,West,Mountain
2,16,Idaho,ID,West,Mountain
3,30,Montana,MT,West,Mountain
4,32,Nevada,NV,West,Mountain


In [27]:
df_merge = dfs.merge(df_state, left_on="STATE", right_on='STATE', how='left')

In [28]:
df_merge['SEX_NM'] = df_merge['SEX'].map({1:'Male', 2:'Female'})

,AGEP,CIT,COW,ENG,ESR,FFSP,LANX,MAR,MULTG,NOC,OCCP,PINCP,POVPIP,RAC1P,SCH,SCHG,SCHL,SCIENGP,SCIENGRLP,SEMP,SEX,WAGP,WKEXREL,WKL,WRK,STATE,STATE_NM,ABBV,REGION,DIVISION,SEX_NM
0,49,1,0,0,6,0,2,3,1,2,9,27800,136,1,1,0,19,0,0,0,1,0,12,3,2,26,Michigan,MI,Midwest,East North Central,Male
1,70,1,0,0,6,0,2,2,1,0,9,38000,442,1,1,0,13,0,0,0,1,0,12,3,2,42,Pennsylvania,PA,Northeast,Middle Atlantic,Male
2,58,1,1,0,1,0,2,1,1,0,410,89000,501,1,1,0,16,0,0,0,1,89000,1,1,1,36,New York,NY,Northeast,Middle Atlantic,Male
3,76,1,0,0,6,0,2,2,1,0,9,96400,501,1,1,0,21,2,2,0,1,0,0,3,2,17,Illinois,IL,Midwest,East North Central,Male
4,57,1,0,0,6,0,2,3,1,0,9,9700,73,1,1,0,16,0,0,0,1,0,0,3,2,39,Ohio,OH,Midwest,East North Central,Male


In [30]:
df_merge['RACE'] = df_merge['RAC1P'].map({1:'White', 2:'Black or African American', 3:'American Indian', 4:'Alaska Native', 5:'American Indian and Alaska Native', 6:'Asian', 7:'Native Hawaiian and Other Pacific Islander', 8:'Some Other Race', 9:'Two or More Races'})

In [33]:
df_merge['EDATTAIN'] = df_merge['SCHL'].map({1:'No schooling completed', 2:'Nursery school, preschool', 3:'Kindergarten', 4:'Grade 1', 5:'Grade 2', 6:'Grade 3', 7:'Grade 4', 8:'Grade 5', 9:'Grade 6', 10:'Grade 7', 11:'Grade 8', 12:'Grade 9', 13:'Grade 10', 14:'Grade 11', 15:'12th grade - no diploma', 16:'Regular high school diploma', 17:'GED or alternative credential',	18:'Some college, but less than 1 year', 19:'1 or more years of college credit, no degree', 20:'Associates degree', 21:'Bachelors degree', 22:'Masters degree', 23:'Professional degree beyond a bachelors degree', 24:'Doctorate degree'})

In [35]:
print(df_merge.shape)
df_merge.head()

(100000, 33)


,AGEP,CIT,COW,ENG,ESR,FFSP,LANX,MAR,MULTG,NOC,OCCP,PINCP,POVPIP,RAC1P,SCH,SCHG,SCHL,SCIENGP,SCIENGRLP,SEMP,SEX,WAGP,WKEXREL,WKL,WRK,STATE,STATE_NM,ABBV,REGION,DIVISION,SEX_NM,RACE,EDATTAIN
0,49,1,0,0,6,0,2,3,1,2,9,27800,136,1,1,0,19,0,0,0,1,0,12,3,2,26,Michigan,MI,Midwest,East North Central,Male,White,"1 or more years of college credit, no degree"
1,70,1,0,0,6,0,2,2,1,0,9,38000,442,1,1,0,13,0,0,0,1,0,12,3,2,42,Pennsylvania,PA,Northeast,Middle Atlantic,Male,White,Grade 10
2,58,1,1,0,1,0,2,1,1,0,410,89000,501,1,1,0,16,0,0,0,1,89000,1,1,1,36,New York,NY,Northeast,Middle Atlantic,Male,White,Regular high school diploma
3,76,1,0,0,6,0,2,2,1,0,9,96400,501,1,1,0,21,2,2,0,1,0,0,3,2,17,Illinois,IL,Midwest,East North Central,Male,White,Bachelors degree
4,57,1,0,0,6,0,2,3,1,0,9,9700,73,1,1,0,16,0,0,0,1,0,0,3,2,39,Ohio,OH,Midwest,East North Central,Male,White,Regular high school diploma


In [37]:
df['SEX'].value_counts(normalize=True)

2    0.509706
1    0.490294
Name: SEX, dtype: float64

In [36]:
# save as csv to be used in Tableau
df_merge.to_csv("./data/tableau.csv", index=False)

### Data Dictionary
Features to be used:

| Feature   | Type | Numerical or Categorical | Description                                                                           |
|-----------|------|--------------------------|---------------------------------------------------------------------------------------|
| AGEP      | int  | N                        | Age: 1 to 99 years                                                                    |
| CIT       | int  | C                        | Citizenship status                                                                    |
| COW       | int  | C                        | Class of worker                                                                       |
| ENG       | int  | C                        | Ability to speak English                                                              |
| ESR       | int  | C                        | Employment status recode                                                              |
| FFSP      | int  | C                        | Yearly food stamp/Supplemental Nutrition Assistance Program (SNAP) recipient          |
| LANX      | int  | C                        | Language other than English spoken at home                                            |
| MAR       | int  | C                        | Marital Status                                                                        |
| MULTG     | int  | C                        | Multigenerational household                                                           |
| NOC       | int  | C                        | Number of own children in household (unweighted)                                      |
| OCCP      | int  | C                        | Occupation recode for 2018 and later based on 2018 OCC codes                          |
| PINCP     | int  | N                        | Total person's income                                                                 |
| POVPIP    | int  | C                        | Income-to-poverty ratio recode                                                        |
| RAC1P     | int  | C                        | Recoded detailed race code                                                            |
| SCH       | int  | C                        | School Enrollment                                                                     |
| SCHG      | int  | C                        | Grade Level Attending                                                                 |
| SCHL      | int  | C                        | Educational Attainment                                                                |
| SCIENGP   | int  | C                        | Field of degree science and engineering flag - NSF definition                         |
| SCIENGRLP | int  | C                        | Field of degree science and engineering related flag - NSF definition                 |
| SEMP      | int  | C                        | Self-employement income past 12 months                                                |
| SEX       | int  | C                        | Sex                                                                                   |
| WAGP      | int  | N                        | Wages or salary income past 12 months (use ADJINC to adjust WAGP to constant dollars) |
| WKEXREL   | int  | C                        | Work experience of householder and spouse                                             |
| WKL       | int  | C                        | When last worked                                                                      |
| WRK       | int  | C                        | Worked last week                                                                      |
| STATE     | int  | C                        | State code based on 2010 Census Definition                                            |